In [2]:
#Memasang library yang dibutuhkan
import sys
import psycopg2
import pandas as pd

In [3]:
#Koneksikan ke database
conn = psycopg2.connect(host="localhost", port=5432, database="dsda", user="postgres", password="12345")
conn.set_session(autocommit = True)
cur = conn.cursor()
sql = "SELECT * FROM orders"
cur.execute(sql)
baris = cur.fetchall()

In [4]:
#memasukkan dataset pada order dan melakukan rename kolumn. Lalu menampilkan data 4 pertama
order = pd.DataFrame([[ij for ij in i]for i in baris])
order.rename(columns={0:'order_id',1:'customer_id',2:'campaign_id',3:'order_date',4:'city',5:'state',6:'zipcode',7:'payment_type',8:'total_price',9:'num_order',10:'num_units'},inplace=True)
order.head(4)

,order_id,customer_id,campaign_id,order_date,city,state,zipcode,payment_type,total_price,num_order,num_units
0,1002854,45978,2141,2009-10-13,NEWTON,MA,2459,VI,190.00,3,3
1,1002855,125381,2173,2009-10-13,NEW ROCHELLE,NY,10804,VI,10.00,1,1
2,1002856,103122,2141,2011-06-02,MIAMI,FL,33137,AE,35.22,2,2
3,1002857,130980,2173,2009-10-14,E RUTHERFORD,NJ,7073,AE,10.00,1,1


In [5]:
#Melakukan cek nilai null
print(order.isnull().sum())

order_id           0
customer_id        0
campaign_id        0
order_date         0
city              17
state           1274
zipcode          320
payment_type     186
total_price      186
num_order        186
num_units        186
dtype: int64


In [6]:
#Membetulkan data yang null dan mengecek adanya null
cleandata = order.dropna()
print(cleandata.isnull().sum())

order_id        0
customer_id     0
campaign_id     0
order_date      0
city            0
state           0
zipcode         0
payment_type    0
total_price     0
num_order       0
num_units       0
dtype: int64


In [7]:
#Menampilkan ordo dari variable order
order.shape

(192983, 11)

In [8]:
#Menampilkan ordo dari variable cleandata
cleandata.shape

(191693, 11)

In [9]:
#menampilkan data 4 pertama
cleandata.head(4)

,order_id,customer_id,campaign_id,order_date,city,state,zipcode,payment_type,total_price,num_order,num_units
0,1002854,45978,2141,2009-10-13,NEWTON,MA,2459,VI,190.00,3,3
1,1002855,125381,2173,2009-10-13,NEW ROCHELLE,NY,10804,VI,10.00,1,1
2,1002856,103122,2141,2011-06-02,MIAMI,FL,33137,AE,35.22,2,2
3,1002857,130980,2173,2009-10-14,E RUTHERFORD,NJ,7073,AE,10.00,1,1


In [10]:
#memanipulasi berdasarkan grup data
data = order[['order_id','customer_id','campaign_id','total_price']]
idOrder = data.groupby('campaign_id')['customer_id'].nunique()
print('Jumlah Customer:\n',idOrder)


Jumlah Customer:
 campaign_id
2001        5
2002       91
2003      276
2004        3
2005       22
        ...  
2235      990
2236    50760
2237     1129
2238        1
2239        4
Name: customer_id, Length: 239, dtype: int64


In [11]:
#memanipulasi berdasarkan filter data NY
man2 = cleandata[['order_id','customer_id','campaign_id','payment_type']]
stateNY = man2['campaign_id'].loc[cleandata['state']=="NY"]
print(stateNY.value_counts())


2141    22687
2173    11272
2236    10606
2010     1460
2237     1183
        ...  
2029        1
2118        1
2132        1
2147        1
2098        1
Name: campaign_id, Length: 182, dtype: int64


In [12]:
#Manipulasi berdasarkan filter data PA
statePA = man2['campaign_id'].loc[cleandata['state']=="PA"]
print(statePA.value_counts())


2141    2523
2236    1984
2173    1644
2010     159
2204     127
        ... 
2145       1
2234       1
2174       1
2113       1
2161       1
Name: campaign_id, Length: 99, dtype: int64


In [13]:
#Normalisasi Zscore (Sebelum)
zdata = cleandata.loc[:,['total_price']]
zdata['total_price'] = zdata['total_price'].fillna(cleandata.groupby('campaign_id')['total_price'].transform('mean'))
print('Data sebelum normalisasi ZScore:\n',zdata)


Data sebelum normalisasi ZScore:
         total_price
0            190.00
1             10.00
2             35.22
3             10.00
4             10.00
...             ...
192978        23.96
192979        20.65
192980        16.95
192981        22.95
192982        49.45

[191693 rows x 1 columns]


In [14]:
#Normalisasi Zscore (Sesudah)
avgdata = zdata.mean()
stddata = zdata.std()
zdata = (zdata-avgdata)/stddata
print('Data setelah normalisasi ZScore:\n',zdata)


Data setelah normalisasi ZScore:
         total_price
0          0.652482
1         -0.333432
2         -0.195294
3         -0.333432
4         -0.333432
...             ...
192978    -0.256968
192979    -0.275098
192980    -0.295364
192981    -0.262501
192982    -0.117352

[191693 rows x 1 columns]


In [15]:
#Normalisasi Minmax (Sebelum)
mdata = cleandata.loc[:,['total_price']]
mdata['total_price'] = mdata['total_price'].fillna(cleandata.groupby('campaign_id')['total_price'].transform('mean'))
print('Data sebelum normalisasi minmax:\n',mdata)


Data sebelum normalisasi minmax:
         total_price
0            190.00
1             10.00
2             35.22
3             10.00
4             10.00
...             ...
192978        23.96
192979        20.65
192980        16.95
192981        22.95
192982        49.45

[191693 rows x 1 columns]


In [16]:
#Normalisasi Minmax (Sesudah)
min = 0
max = 1
mindata = mdata.min()
maxdata = mdata.max()
mdata = ((mdata-mindata)*(max-min)/(maxdata-mindata))+min
print('Data setelah normalisasi minmax:\n',mdata)


Data setelah normalisasi minmax:
         total_price
0          0.019291
1          0.001015
2          0.003576
3          0.001015
4          0.001015
...             ...
192978     0.002433
192979     0.002097
192980     0.001721
192981     0.002330
192982     0.005021

[191693 rows x 1 columns]


In [17]:
cleandata2 = cleandata[cleandata.payment_type != "??"]
cleandata2.shape

(191382, 11)

In [18]:
data2 = cleandata2[['order_id','customer_id','campaign_id','total_price','payment_type']]
paymentOrder = data2.groupby('payment_type')['customer_id'].nunique()
print('Jumlah Customer:\n',paymentOrder)


Jumlah Customer:
 payment_type
AE    46678
DB    12412
MC    46447
OC     8155
VI    75042
Name: customer_id, dtype: int64


In [19]:
data3 = cleandata2[['order_id','customer_id','campaign_id','city','total_price']]
cityOrder = data3.groupby('city')['customer_id'].nunique()
print('Jumlah Customer:\n',cityOrder)

Jumlah Customer:
 city
.EASTHAM               2
1083 EA  AMSTERDAM     1
1660 MISSION           1
1NGLEWOOD              1
2130                   1
                      ..
ZIEGLERVILLE           1
ZIONSDILLE             2
ZIONSVILLE            22
ZIONSVIOLE             1
ZOLLIKERBERG           2
Name: customer_id, Length: 12076, dtype: int64


In [20]:
man3 = cleandata2[['order_id','customer_id','campaign_id','payment_type','num_units']]
stateTX = man3['num_units'].loc[cleandata2['state']=="TX"]
print('Banyaknya pesanan order berdasarkan unit di state TX:\n', stateTX.value_counts())

Banyaknya pesanan order berdasarkan unit di state TX:
 1      4107
2      1249
3       532
4       175
5        75
6        30
7        24
10       16
8         7
11        6
9         3
15        3
13        2
20        2
25        1
16        1
101       1
Name: num_units, dtype: int64


In [21]:
man4 = cleandata2[['order_id','customer_id','campaign_id','payment_type','num_order']]
stateFL = man4['num_order'].loc[cleandata2['state']=="FL"]
print('Banyaknya order berdasarkan orderlines dari State FL:\n', stateFL.value_counts())

Banyaknya order berdasarkan orderlines dari State FL:
 1     7336
2     1681
3      695
4      198
5       99
6       59
7       31
8       18
9       15
11       7
10       4
12       3
22       2
14       1
20       1
13       1
Name: num_order, dtype: int64
